OVERVIEW - this Notebook will be used to complete the IBM Data Science Certificate Capstone Project. Project will include 3 sections as follows (i) Scraping Data from a Wikipedia Page (ii) Establishing a dataframe which includes location data for neigborhoods such as latitudes and longitudes  (iii) performing clustering/segmentation of the neighborhood and creating map visualizations

In [2]:
# importing initial python libraries
import pandas as pd
import numpy as np
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


# PART I - SCRAPING DATA FROM WIKIPEDIA PAGE

In [3]:
#installing necessary packages and the Requests Library. Using BeautifulSoup to scrape the wikipage
!pip install bs4
import requests
from bs4 import BeautifulSoup

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=c0905da204025b1bfe80365230b2289af78a35e724e003ee46759b5e8aa0d203
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


The wiki page 'https://wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' contains list of postal codes. Postal codes beginning with M are located within the city of Toronto in the province of Ontario.

In [4]:
#downloading source code for wiki page and creating the soup object
url= 'https://wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url)
soup = BeautifulSoup(html_data.content)
soup.title

<title>List of postal codes of Canada: M - Wikipedia</title>

Parsing through the web page code and extracting the relevant information which is contained in the class attributes ('td'), skipping rows that do not have assigned postalcodes

In [5]:

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)

df.shape

(103, 3)

In [8]:
#cleaning up some data for boroughs that have unusual names
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [9]:
df.tail(20)

,PostalCode,Borough,Neighborhood
83,M4T,Central Toronto,"Moore Park, Summerhill East"
84,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park"
85,M1V,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo..."
86,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest..."
87,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."
88,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores"
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
90,M1W,Scarborough,"Steeles West, L'Amoreaux West"
91,M4W,Downtown Toronto,Rosedale
92,M5W,Downtown Toronto Stn A,Enclave of M5E


In [32]:
df.head(20)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [17]:
#size of dataframe
df.shape

(103, 3)

# PART II - IMPORTING GEOSPATIAL INFORMATION FOR ALL NEIGHBORHOODS#

In [14]:
#import required packages
!pip install geopy
from geopy.geocoders import Nominatim


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [15]:
geolocator=Nominatim(user_agent="foursquare_agent")


In [29]:

import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

ModuleNotFoundError: No module named 'geocoders'

In [43]:
#Downloading data from a link to a file containing coordinates for all postal codes in Toronto
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv
locationData = "Geospatial_Coordinates.csv"

--2021-06-03 03:31:15--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2788 (2.7K) [text/csv]
Saving to: ‘Geospatial_Coordinates.csv’

Geospatial_Coordina 100%[===================>]   2.72K  --.-KB/s    in 0s      

2021-06-03 03:31:15 (63.4 MB/s) - ‘Geospatial_Coordinates.csv’ saved [2788/2788]



In [46]:
#open, read and print contents of the CSV file
with open ("Geospatial_Coordinates.csv",'r') as file1:
    print(file1.read())

Postal Code,Latitude,Longitude
M1B,43.8066863,-79.1943534
M1C,43.7845351,-79.1604971
M1E,43.7635726,-79.1887115
M1G,43.7709921,-79.2169174
M1H,43.773136,-79.2394761
M1J,43.7447342,-79.2394761
M1K,43.7279292,-79.2620294
M1L,43.7111117,-79.2845772
M1M,43.716316,-79.2394761
M1N,43.692657,-79.2648481
M1P,43.7574096,-79.273304
M1R,43.7500715,-79.2958491
M1S,43.7942003,-79.2620294
M1T,43.7816375,-79.3043021
M1V,43.8152522,-79.2845772
M1W,43.7995252,-79.3183887
M1X,43.8361247,-79.2056361
M2H,43.8037622,-79.3634517
M2J,43.7785175,-79.3465557
M2K,43.7869473,-79.385975
M2L,43.7574902,-79.3747141
M2M,43.789053,-79.4084928
M2N,43.7701199,-79.4084928
M2P,43.7527583,-79.4000493
M2R,43.7827364,-79.4422593
M3A,43.7532586,-79.3296565
M3B,43.7459058,-79.352188
M3C,43.7258997,-79.340923
M3H,43.7543283,-79.4422593
M3J,43.7679803,-79.4872619
M3K,43.7374732,-79.4647633
M3L,43.7390146,-79.5069436
M3M,43.7284964,-79.4956974
M3N,43.7616313,-79.5209994
M4A,43.7258823,-79.3155716
M4B,43.7063972,-79.309937
M4C,43

In [49]:
#Convert CSV file into Panda Dataframe
df2=pd.read_csv(locationData)
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [65]:
#sort the original dataframe (df) by PostalCode so it is consistent with Df2
df_sorted=df.sort_values("PostalCode").reset_index()
df_sorted.head()
df_sorted.shape

(103, 4)

In [66]:
#merge the two dataframes and drop the original index column
df_merged=df_sorted
df_merged[['Latitude','Longitude']]=df2[['Latitude','Longitude']]
df_merged.drop('index', axis=1, inplace=True)
df_merged.tail()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
102,M9W,Etobicoke Northwest,"Clairville, Humberwood, Woodbine Downs, West H...",43.706748,-79.594054
